In [1]:
import TestGAN_module
from TestGAN_module import *
import Util

import os
import pandas as pd
import numpy as np
import torch

In [2]:
# CSV path
folder_name = "data"
file_name = "hungary_chickenpox.csv"
path = os.path.join(folder_name, file_name)
#Load CSV into Dataframe
df = pd.read_csv(path,sep=',')
df = df.drop(columns = ['Date'])

In [3]:
# Get column names
names = Util.get_column_names(df)

In [4]:
df

,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,HEVES,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
0,168,79,30,173,169,42,136,120,162,36,130,57,2,178,66,64,11,29,87,68
1,157,60,30,92,200,53,51,70,84,28,80,50,29,141,48,29,58,53,68,26
2,96,44,31,86,93,30,93,84,191,51,64,46,4,157,33,33,24,18,62,44
3,163,49,43,126,46,39,52,114,107,42,63,54,14,107,66,50,25,21,43,31
4,122,78,53,87,103,34,95,131,172,40,61,49,11,124,63,56,7,47,85,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,95,12,41,6,39,0,16,15,14,10,56,7,13,122,4,23,4,11,110,10
518,43,39,31,10,34,3,2,30,25,19,34,20,18,70,36,5,23,22,63,9
519,35,7,15,0,0,0,7,7,4,2,30,36,4,72,5,21,14,0,17,10
520,30,23,8,0,11,4,1,9,10,17,27,17,21,12,5,17,1,1,83,2


In [5]:
# Set random state
torch.manual_seed(111)
# Select # of data points from data set to train GAN
num_data = 450
# Specify Validation Data
val_data = 50
# Specify dimensionality of dataframe
df_dim = len(df.columns)
# learning rate
lr = 0.002
# number of epoch
num_epochs = 1000
# Discriminator and Generator dropout fro data standardization
drop_out=0.3
# Define batch size for the data loader
batch_size = 25


# These create the arrays and then turns them into tensors for the train loader 
arrays = Util.create_arrays(df[:num_data],num_data)
train_set = Util.create_tensors(arrays)
# val set
val_arrays = Util.create_arrays(df[num_data:],val_data)
val_set = Util.create_tensors(val_arrays)
# Define discriminator and generator for training function
discriminator = Discriminator(df_dim=df_dim, batch_size=batch_size, drop_out=drop_out)
generator = Generator(df_dim=df_dim, batch_size=batch_size)

In [6]:
# The train loader loads the data into the neural network trainer
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size)

In [7]:
# This takes the trining dataframe as input
# training_loop(num_epochs, train_loader, lr)
# This takes the entire dataframe as input (Val Data Only)
training_loop(num_epochs=num_epochs, train_loader=train_loader, val_loader=val_loader, df_dim=df_dim, batch_size=batch_size, drop_out=drop_out, lr=lr, early_stopping_patience=10)

TypeError: Generator.__init__() got multiple values for argument 'df_dim'